In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')
%cd  /content/drive/'My Drive'/dataset/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/dataset


In [ ]:


import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import cv2 
import glob 

batch_size = 32
momentum = 0.9
lr = 0.001
epochs = 10
log_interval = 10



X = []
y = []

for i in glob.glob('./fake_face_dcgan/*'):
  im = cv2.imread(i)
  X.append(im)
  y.append(1)

for i in glob.glob('./real_face/*'):

  im = cv2.imread(i)
  im = cv2.resize(im, (64,64), interpolation = cv2.INTER_AREA)
  X.append(im)
  y.append(0)



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

X_train, X_test, y_train, y_test = torch.tensor(X_train), torch.tensor(X_test), torch.tensor(y_train), torch.tensor(y_test) 

X_train = X_train.permute(0, 3, 1, 2)

X_test = X_test.permute(0, 3, 1, 2)

X_train.shape


my_dataset_train = TensorDataset(X_train, y_train) # create your datset
train_loader = DataLoader(my_dataset_train, batch_size=32, shuffle=True)

my_dataset_test = TensorDataset(X_test, y_test) # create your datset
test_loader = DataLoader(my_dataset_test, batch_size=32, shuffle=True)




In [ ]:
X_test.shape

torch.Size([2310, 3, 64, 64])

In [ ]:
import numpy as np
np.unique(y)

array([0, 1])

In [ ]:
i = iter(train_loader)
x1,y1 = next(i)
x1.shape

torch.Size([32, 3, 64, 64])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

nclasses = 2 

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, kernel_size=3)
        self.batchnorm1 = nn.BatchNorm2d(20)
        self.conv2 = nn.Conv2d(20, 20, kernel_size=3)
        self.batchnorm2 = nn.BatchNorm2d(20)
        self.conv3 = nn.Conv2d(20, 20, kernel_size=3)
        self.batchnorm3 = nn.BatchNorm2d(20)
        self.conv4 = nn.Conv2d(20, 20, kernel_size=3)
        self.batchnorm4 = nn.BatchNorm2d(20)
        # self.conv5 = nn.Conv2d(20, 20, kernel_size=3)
        # self.batchnorm5 = nn.BatchNorm2d(20)
        
        # self.conv2_drop = nn.Dropout2d()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(62720, 100)
        self.fc2 = nn.Linear(100, nclasses)

        for m in self.modules():
          if isinstance(m, nn.Conv2d):
              torch.nn.init.xavier_uniform(m.weight) #, nonlinearity='relu'
          elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
              nn.init.constant_(m.weight, 1)
              nn.init.constant_(m.bias, 0)

    def forward(self, x):
        relu = nn.ReLU()

        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = relu(x)
        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = relu(x)
        x = self.conv3(x)
        # x = F.max_pool2d(x, 2)
        x = self.batchnorm3(x)
        x = relu(x)
        x = self.conv4(x)
        x = self.batchnorm4(x)
        x = relu(x)

        # x = self.conv5(x)
        # x = F.max_pool2d(x, 2)
        # x = self.batchnorm5(x)
        x = relu(x)
        # x = drop_
        x = x.view(-1, 62720)
        # x = self.flatten(x)
        x = relu(self.fc1(x))
        # x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x,dim=1)


# Training

In [ ]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = GoogLeNet().to(device)
model = Net().to(device)


# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    correct =0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device).float(), target.to(device).float()
        
        optimizer.zero_grad()
   
        output = model(data)
        # print(data.shape)
        # print(target.shape)
        # print(output.shape)

        loss = loss_func(output, target.long())
        loss.backward()
        optimizer.step()
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        # if batch_idx % log_interval == 0:
    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy{:.3f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item(), 100. * correct / len(train_loader.dataset) ))

def validation():
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.to(device).float()
        target = target.to(device).long()
        output = model(data)
        validation_loss += F.nll_loss(output, target, reduction="sum").item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    validation_loss /= len(test_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        validation_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, epochs + 1):
    train(epoch)
    validation()
    model_file = 'model_' + str(epoch) + '.pth'
    torch.save(model.state_dict(), model_file)
    print('\nSaved model to ' + model_file + '.')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


Train Epoch: 1 [2184/5389 (99%)]	Loss: 0.000335	Accuracy97.940

Validation set: Average loss: 0.0051, Accuracy: 2305/2310 (100%)


Saved model to model_1.pth.
Train Epoch: 2 [2184/5389 (99%)]	Loss: 0.000406	Accuracy99.963

Validation set: Average loss: 0.0061, Accuracy: 2306/2310 (100%)


Saved model to model_2.pth.
Train Epoch: 3 [2184/5389 (99%)]	Loss: 0.000004	Accuracy99.963

Validation set: Average loss: 0.0031, Accuracy: 2307/2310 (100%)


Saved model to model_3.pth.
Train Epoch: 4 [2184/5389 (99%)]	Loss: 0.000000	Accuracy100.000

Validation set: Average loss: 0.0022, Accuracy: 2308/2310 (100%)


Saved model to model_4.pth.
Train Epoch: 5 [2184/5389 (99%)]	Loss: 0.000001	Accuracy100.000

Validation set: Average loss: 0.0019, Accuracy: 2308/2310 (100%)


Saved model to model_5.pth.
Train Epoch: 6 [2184/5389 (99%)]	Loss: 0.000001	Accuracy100.000

Validation set: Average loss: 0.0017, Accuracy: 2308/2310 (100%)


Saved model to model_6.pth.
Train Epoch: 7 [2184/5389 (99%)]	Loss: 0.00